In [1]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [2]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    outvec = np.zeros_like(vec)
    outvec[0] = np.linalg.norm(vec)
    u = (vec - outvec) / np.linalg.norm(vec - outvec)
    H = np.eye(u.size) - 2 * np.outer(u, u)
    return outvec, H

Test your function using tests below:

In [3]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v, atol=1e-10)
assert_allclose(np.dot(h, v), v1, atol=1e-10)

In [4]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [5]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    Q = np.eye(m)
    for i in range(n):
        vec, H_temp = householder(a1[i:, i])
        zeros_h = np.zeros((i, m - i))
        H = np.asarray(np.bmat([[np.eye(i), zeros_h], [zeros_h.T, H_temp]]))
        a1 = H @ a1
        Q = Q @ H.T
    return Q, a1

In [6]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [7]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)
print(r)

[[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.         -0.          0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.         -0.         -0.        ]]


Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [8]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

In [9]:
print(q - qq)
print(r - rr)

[[ 0.27330099  0.         -0.18739504  0.1080946   0.80208706]
 [ 1.1207179   0.         -1.06653761  0.54316773 -0.28376255]
 [ 0.39451844 -0.          1.20136927  0.05494342 -0.39374214]
 [ 1.24996836  0.          1.04289376  0.09486041  0.26573876]
 [ 0.97531136 -0.         -0.54448611 -0.79823623 -0.0799949 ]]
[[ 2.80305537  2.5028758   1.7904723 ]
 [ 0.          0.         -0.        ]
 [ 0.         -0.          1.09920919]
 [ 0.          0.          0.        ]
 [ 0.         -0.         -0.        ]]


*Enter your explanation here* (10% of the total grade, peer-graded)

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and 

provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).


Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [10]:
#   All of the following is the creation of Gram-Schmidt algorithm for decomposition. Seems not to be necessary lmao :c
def projection(vec, u):
    if (vec.ndim != 1) or (u.ndim != 1):
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    return (u @ vec) / (u @ u) * u

In [11]:
def gmsh(a1):
    '''
    docstrings r for losers
    '''
    a = np.array(a1, copy=True, dtype=float)
    n, m = a.shape  #   n is for nof rows, m iis for nof cols
    eigen_vecs = []
    a_array = []
    for i in range(m):
        u = a[:, i].flatten()
        for j in range(i):
            u = u - projection(u, eigen_vecs[j])
        eigen_vecs.append(u / np.linalg.norm(u))



    #   Filling in Q matrix
    Q = np.empty((n, n))
    for i in range(len(eigen_vecs)):
        Q[:, i] = eigen_vecs[i]

    #   Filling in R matrix
    R = np.zeros_like(a)
    for j in range(m):
        a_vec = a[:, j].flatten()
        for i in range(j + 1):
            R[i, j] = eigen_vecs[i] @ a_vec
            
    return Q, R

In [12]:
# Test III.1

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
#a = np.array([[12, -51, 4], [6, 167, -68], [-4, 24, -41]])
#a = np.array([[2, -2, 18], [2, 1, 0], [1, 2, 0]])
q, r = gmsh(a)

# test the decomposition itself
assert_allclose(np.dot(q, r), a, atol=1e-10)
